<a href="https://colab.research.google.com/github/nadare881/voice-changer-vector-search/blob/develop/notebook/search_similar_voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 概要
本notebookでは音声ファイルから類似したデータセット・モデルの検索を行います

`ランタイム > ランタイムのタイプを変更 > ハードウェアアクセラレータ` から`GPU`を選択してください。

# 準備コード
## ライブラリのインストール

In [1]:
!pip install -qq https://github.com/pyannote/pyannote-audio/archive/refs/heads/develop.zip

     | 16.6 MB 5.3 MB/s 0:00:04
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## リポジトリのクローン + DBの整備

In [2]:
!git clone https://github.com/nadare881/voice-changer-vector-search.git

Cloning into 'voice-changer-vector-search'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 84 (delta 27), reused 69 (delta 19), pack-reused 0
Unpacking objects: 100% (84/84), 305.02 KiB | 1.01 MiB/s, done.


In [3]:
import numpy as np
import pandas as pd

source_embedding = np.load("./voice-changer-vector-search/data/embeddings.npy")
meta_df = pd.read_csv("./voice-changer-vector-search/data/meta.csv", encoding="utf-8")

## modelのロード
必要なもの: 

- HuggingFaceのアカウント + token
- https://huggingface.co/pyannote/embedding のユーザー規約への同意

In [4]:
from pyannote.audio import Model, Inference
# ここにtokenを入力してください
token = "your-huggingface-token"
model = Model.from_pretrained("pyannote/embedding", 
                              use_auth_token=token)
inference = Inference(model, window="whole", device="cuda")

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.0.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../root/.cache/torch/pyannote/models--pyannote--embedding/snapshots/20b2db779562a3141f5eadd34a0232dbcd56d620/pytorch_model.bin`
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.0.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../root/.cache/torch/pyannote/models--pyannote--embedding/snapshots/20b2db779562a3141f5eadd34a0232dbcd56d620/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.0.0+cu118. Bad things might happen unless you revert torch to 1.x.
Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.0.0+cu118. Bad things might happen unless you revert torch to 1.x.


# データベースの検索

## 音声からembddingの抽出
google colaboratoryを用いる際は左のフォルダのマークのところからファイルをアップロードしてください



In [6]:
import os
os.listdir("./")

['.config',
 'voice-changer-vector-search',
 'sample.wav',
 '.ipynb_checkpoints',
 'sample_data']

In [7]:
query_embedding = inference("sample.wav")
query_embedding /= np.maximum(1e-9, np.linalg.norm(query_embedding, ord=2))

In [9]:
score = np.einsum("d,nd->n", query_embedding, source_embedding)
order = np.argsort(-score)

res_df = meta_df.iloc[order, :].reset_index(drop=True)
res_df["score"] = score[order]
res_df

,voice_id,display_name,distribution_type,url,voice_sample,thumbnail,speaker,actor,corpus,language,...,license_url,terms_link,commercial_use,price,twitter,ex_url_1,model_type,base_model,input_voice,score
0,0,あみたろのITAコーパス読み上げ音声,voice,https://amitaro.net/voice/corpus-list/ita/,https://www.youtube.com/embed/ksKu6JyLP5I,https://amitaro.net/wp-content/uploads/cfc421f...,あみたろ,あみたろ,ITA,ja,...,https://amitaro.net/voice/corpus-list/ita/,https://amitaro.net/voice/voice_rule/,True,0,https://twitter.com/amitaro_utau,https://amitaro.net/,NaN,NaN,NaN,0.664590
1,3,黄琴海月 ITAコーパス読み上げ音声【4.00】,voice,https://kikyohiroto1227.wixsite.com/kikoto-uta...,https://www.youtube.com/embed/SXnB8b1fKP4,https://static.wixstatic.com/media/8a7d85_5bec...,黄琴海月,黄鏡博人,ITA,ja,...,https://kikyohiroto1227.wixsite.com/kikoto-uta...,https://kikyohiroto1227.wixsite.com/kikoto-uta...,True,0,https://twitter.com/KikyoHiloto,NaN,NaN,NaN,NaN,0.463548
2,3,黄琴まひろ ITAコーパス読み上げ音声【1.00】,voice,https://kikyohiroto1227.wixsite.com/kikoto-uta...,https://www.youtube.com/embed/LT6D4Mx2xgo,https://static.wixstatic.com/media/8a7d85_767d...,黄琴まひろ,黄鏡博人,ITA,ja,...,https://kikyohiroto1227.wixsite.com/kikoto-uta...,https://kikyohiroto1227.wixsite.com/kikoto-uta...,True,0,https://twitter.com/KikyoHiloto,NaN,NaN,NaN,NaN,0.335240
3,2,刻鳴時雨ITAコーパス読み上げ音声素材,voice,https://booth.pm/ja/items/3640133,NaN,https://s2.booth.pm/4bbcead3-a3a3-40b7-8364-82...,刻鳴時雨,丸ころ,ITA,ja,...,https://bindume-chan.booth.pm/items/3640133,https://bindume-chan.booth.pm/items/3640133,True,0,NaN,NaN,NaN,NaN,NaN,0.177222
4,1,刻鳴時雨 RVC用学習済みモデル,model,https://huggingface.co/yasyune/Shigure_Tokina_RVC,NaN,https://s2.booth.pm/4bbcead3-a3a3-40b7-8364-82...,刻鳴時雨,丸ころ,ITA,ja,...,https://bindume-chan.booth.pm/items/3640133,https://bindume-chan.booth.pm/items/3640133,True,0,NaN,NaN,RVC,hubert_basr,https://bindume-chan.booth.pm/items/3640133,0.141767


## pretty display

In [10]:
def handle_thumbnail(x, width=200):
    if pd.isna(x) or type(x) != str:
        return np.NaN
    return  "<img src='{}' width='{}' />".format(x, width)

def handle_voicesample(x):
    if pd.isna(x) or type(x) != str:
        return np.NaN
    if x.split(".")[-1] in ["mp3", "wav"]:
        return '<audio controls><source src="{}"></audio>'.format(x)
    if x.startswith("https://www.youtube.com"):
        html = """<iframe width="560" height="315" src="{}" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>""".format(x)
        return html

In [11]:
from IPython.display import HTML
display_df = res_df[["display_name", "url", "score"]].head(5)
display_df["thumbnail"] = res_df["thumbnail"].map(handle_thumbnail)
display_df["voice_sample"] = res_df["voice_sample"].map(handle_voicesample)

HTML(display_df.to_html(classes=["table", "table-bordered", "table-hover"], escape=False))


,display_name,url,score,thumbnail,voice_sample
0,あみたろのITAコーパス読み上げ音声,https://amitaro.net/voice/corpus-list/ita/,0.664590,,
1,黄琴海月 ITAコーパス読み上げ音声【4.00】,https://kikyohiroto1227.wixsite.com/kikoto-utau/kurage,0.463548,,
2,黄琴まひろ ITAコーパス読み上げ音声【1.00】,https://kikyohiroto1227.wixsite.com/kikoto-utau/mahiro,0.335240,,
3,刻鳴時雨ITAコーパス読み上げ音声素材,https://booth.pm/ja/items/3640133,0.177222,,NaN
4,刻鳴時雨 RVC用学習済みモデル,https://huggingface.co/yasyune/Shigure_Tokina_RVC,0.141767,,NaN
